**----------------------------------------------------------------------------------------------------**
# **DATA EXPLORATION**
**----------------------------------------------------------------------------------------------------**

The **data_explorer.py** file is a personal python script in which I use to perform initial exploration of datasets. It uses a function to give key summary features of common data file types. More details on this can be found in the Dataset_Explorer Repository on my Github profile.

In [2]:
# IMPORT DATA EXPLORER PYTHON FILE TO ANALYSE EACH DATASET

import data_explorer

In [5]:
data_explorer.analyze_dataset('data_extracts/accounts.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
7,200,0,"11,328"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,account_id,int64,0,0,200,100.5,200.0,1.0,0,0,0
1,company_name,object,0,0,200,nan,nan,nan,65,0,0
2,company_address,object,0,0,200,nan,nan,nan,0,0,200
3,contact_person,object,0,0,200,nan,nan,nan,0,0,0
4,contact_phone,object,0,0,200,nan,nan,nan,151,0,0
5,gender,object,0,0,2,nan,nan,nan,0,0,0
6,joining_date,object,0,0,179,nan,nan,nan,200,0,0


account_id,company_name,company_address,contact_person,contact_phone,gender,joining_date
34,Davis-Pruitt,"37642 Brittany Pass Apt. 945\nEast Lauren, PA 77160",Emily Castillo,385.799.6437x390,F,2017-07-03
14,"Howard, Vega and Chen","564 White Pike Suite 650\nPort Kenneth, IA 46122",Samantha Jackson,001-639-424-0631,F,2018-12-22
140,Randall-Arias,"5241 Jason Turnpike\nFergusonmouth, HI 39510",William Graves,(364)487-3320x4441,M,2018-07-27
144,Pierce-Smith,USCGC Bryant\nFPO AA 47848,Tiffany Yang,8243479140,F,2019-04-01
106,"Mendoza, Whitaker and Fowler","24715 Mata Locks Apt. 068\nKennethborough, ID 23170",Elizabeth Moreno,(064)209-6594x919,F,2016-08-12
128,"Gonzalez, Williams and Lewis","63230 Julie Cliffs Suite 717\nNew Christopher, NC 94620",Kevin Benjamin,001-167-786-6584x36251,M,2018-12-31
99,Stevenson Inc,Unit 2296 Box 8103\nDPO AE 26923,Ashley Rojas,+1-614-427-7847x9822,F,2019-04-25
6,Mullen PLC,"6012 Elizabeth Islands Suite 689\nSarahshire, WI 09427",Rebecca Mcknight,+1-232-492-4506,F,2019-04-23
7,Curry and Sons,"473 Erik Divide Suite 172\nPort Danielle, ND 05706",Jeffery Harris,+1-520-397-4228x1402,M,2017-08-31
63,Underwood-Rodriguez,"3411 Whitney Street\nTurnerfort, SD 69683",Jeremy Cross,1383386247,M,2018-09-23


In [3]:
data_explorer.analyze_dataset('data_extracts/invoices.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
4,"9,685",0,"310,048"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,invoice_id,int64,0,0,"9,685","4,843.0","9,685.0",1.0,0,0,0
1,account_id,int64,0,0,200,98.4,200.0,1.0,0,0,0
2,date_issued,object,0,0,366,nan,nan,nan,"9,685",0,0
3,payment_dates,object,0,0,394,nan,nan,nan,"9,685",0,0


invoice_id,account_id,date_issued,payment_dates
7199,147,2020-12-22,2021-01-17
4148,83,2020-11-02,2020-11-30
23,1,2021-03-22,2021-04-18
5546,109,2020-12-08,2021-01-03
6551,134,2020-08-11,2020-09-10
2220,45,2020-09-22,2020-10-18
106,3,2020-12-15,2021-01-11
5851,116,2020-09-02,2020-10-13
6136,122,2020-11-12,2020-12-10
8660,177,2020-07-23,2020-09-06


In [4]:
data_explorer.analyze_dataset('data_extracts/invoice_line_items.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
3,"53,481",0,"1,283,672"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,invoice_id,int64,0,0,"9,685","4,847.1","9,685.0",1.0,0,0,0
1,item_id,object,0,0,100,nan,nan,nan,"53,481",0,0
2,quantity,int64,0,0,4,2.5,4.0,1.0,0,0,0


invoice_id,item_id,quantity
4943,1-66068-167-7,1
9599,1-83164-707-9,4
8527,1-01-376732-2,4
4483,0-667-81667-4,1
9345,0-682-99878-8,3
8017,0-493-95641-7,4
4620,1-227-05897-7,3
8045,0-8153-0450-1,3
356,1-83164-707-9,2
1858,1-5437-3256-9,1


In [5]:
data_explorer.analyze_dataset('data_extracts/skus.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
5,100,0,"4,128"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,item_id,object,0,0,100,nan,nan,nan,100,0,0
1,item_name,object,0,0,100,nan,nan,nan,59,0,0
2,item_description,object,0,0,100,nan,nan,nan,63,0,0
3,item_cost_price,float64,0,0,100,"4,871.6","9,956.5",115.7,0,0,0
4,item_retail_price,float64,0,0,100,"10,149.0","28,111.4",239.7,0,0,0


item_id,item_name,item_description,item_cost_price,item_retail_price
1-107-39856-8,Secured transitional open architecture,transition killer mindshare,8675.24,21728.93
0-9878492-7-1,Decentralized didactic throughput,maximize back-end solutions,4418.58,6788.22
1-320-98453-3,Managed value-added neural-net,expedite plug-and-play e-business,4070.52,11391.13
1-354-00803-0,Stand-alone modular knowledge user,evolve value-added systems,1404.94,2316.52
1-67777-184-4,Quality-focused cohesive orchestration,repurpose proactive initiatives,2854.05,5578.74
0-88310-438-5,Centralized needs-based groupware,syndicate best-of-breed action-items,5108.92,8777.13
0-248-44413-1,Profound eco-centric policy,incubate virtual communities,8102.07,20345.94
0-7631-7289-8,Object-based systemic software,synergize dynamic mindshare,8139.64,21575.53
1-5437-3256-9,Sharable maximized migration,envisioneer best-of-breed action-items,8024.52,9627.81
1-71447-910-2,Configurable systematic capability,engage revolutionary platforms,6621.62,19293.98


**----------------------------------------------------------------------------------------------------**
# **PYSPARK ETL Process**
**----------------------------------------------------------------------------------------------------**

In [6]:
# STEP 1: IMPORT REQUIRED LIBRARIES

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [7]:
# STEP 2: CREATE SPARK SESSION

spark = SparkSession.builder.appName("InvoiceETL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/30 19:13:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# STEP 3: LOAD CSV EXTRACTS INTO DATAFRAMES

accounts_df = spark.read.csv("data_extracts/accounts.csv", inferSchema=True, header=True, quote='"', escape="\\", multiLine=True) # USE QUOTE AND MULTILINE FOR company_address
skus_df = spark.read.csv("data_extracts/skus.csv", header=True, inferSchema=True)
invoices_df = spark.read.csv("data_extracts/invoices.csv", header=True, inferSchema=True)
invoice_line_items_df = spark.read.csv("data_extracts/invoice_line_items.csv", header=True, inferSchema=True)

In [9]:
# STEP 4: CREATE TEMP TABLES FOR SPARKSQL

accounts_df.createOrReplaceTempView("accounts")
skus_df.createOrReplaceTempView("skus")
invoices_df.createOrReplaceTempView("invoices")
invoice_line_items_df.createOrReplaceTempView("invoice_line_items")

In [19]:
# STEP 5: TRANSFORM DATAFRAMES INTO FEATURES REQUIRED BY DATA SCIENTISTS

# STEP 5A: CREATE INTERMEDIATE MODEL TO AGGREGATE INVOICE LINE ITEMS

try: 
  
  invoices_total = spark.sql("""

-----------COMMON TABLE EXPRESSION (CTE) 'INVOICES_TOTAL' CALCULATES AGGREGATED DATA FOR EACH INVOICE-----------
---------------------JOINS skus (Stock Keeping Units) INFORMATION INTO invoice_line_items-----------------------
-----------------------------THE CTE IS CREATED TO BE USED LATER IN THE MAIN QUERY------------------------------

  CREATE OR REPLACE TEMPORARY VIEW invoices_total AS (
      SELECT
        l.invoice_id
        ,SUM(l.quantity) AS total_quantity
        ,ROUND(SUM(s.item_cost_price), 2) AS total_cost_price
        ,ROUND(SUM(s.item_retail_price), 2) AS total_retail_price
          
      FROM invoice_line_items l
        LEFT JOIN skus s
        ON l.item_id = s.item_id

      GROUP BY l.invoice_id
      ORDER BY l.invoice_id )

    """)

  spark.sql("SELECT * FROM invoices_total").show()

except Exception as e:
    # RETRIEVE SQL ERROR MESSAGE
    sql_error_message = str(e)
    print("SQL Error Message:", sql_error_message)



+----------+--------------+----------------+------------------+
|invoice_id|total_quantity|total_cost_price|total_retail_price|
+----------+--------------+----------------+------------------+
|         1|            16|        19226.36|          56060.72|
|         2|             6|        16707.95|          34681.89|
|         3|            15|        24764.52|          46466.53|
|         4|            10|        19343.63|          46154.11|
|         5|            21|         52305.2|         120371.88|
|         6|             8|          1045.1|           2446.93|
|         7|            16|        36489.67|          55755.93|
|         8|            15|        15663.26|          34691.34|
|         9|             5|         2513.75|           4700.64|
|        10|            12|        14504.82|          32761.98|
|        11|            17|        25524.91|          56795.14|
|        12|             3|          9246.6|           23659.9|
|        13|            12|        32160

In [20]:
# STEP 5B: FINAL DATAFRAME USING INTERMEDIATE MODEL AND ADDITIONAL CALCULATIONS

try: 
  
  invoices_final = spark.sql("""

------------SELECTS ATTRIBUTES RELATED TO INTERMEDIATE MODEL, INVOICES AND ACCOUNTS-------------
-----------------------------USES invoices AS STARTIUNG POINT-----------------------------------
------------------JOINS invoices_total AND accounts INFORMATION INTO invoices-------------------

  SELECT 
    i.invoice_id AS inv_id
    ,i.account_id AS acct_id
    ,t.total_retail_price AS inv_total
    ,t.total_quantity AS inv_items

    ,DATEDIFF(day, a.joining_date, i.date_issued) AS acct_age       --CALCULATE THE ACCOUNT AGE
    
    ,COUNT(i.invoice_id) OVER (PARTITION BY i.account_id 
      ORDER BY i.date_issued 
      RANGE BETWEEN 120 PRECEDING AND 1 PRECEDING) AS num_inv_120d  --COUNT INVOICES IN THE LAST 120 DAYS FROM DATE ISSUED
    
    ,ROW_NUMBER() OVER (PARTITION BY i.account_id 
      ORDER BY i.date_issued, i.invoice_id) AS cum_tot_inv_acct     --CALCULATE CUMULATIVE INVOICES FOR ACCOUNT

    ,CASE 
      WHEN DATEDIFF(i.payment_dates, i.date_issued) > 30 THEN 1
      ELSE 0
     END AS is_late                                                 --BINARY FLAG IF THE INVOICE IS PAID LATE

  FROM invoices i
    LEFT JOIN invoices_total t
    ON i.invoice_id = t.invoice_id

    LEFT JOIN accounts a
    ON a.account_id = i.account_id

  ORDER BY i.invoice_id


  """)

  invoices_final.show()

except Exception as e:
    # RETRIEVE SQL ERROR MESSAGE
    sql_error_message = str(e)
    print("SQL Error Message:", sql_error_message)




+------+-------+---------+---------+--------+------------+----------------+-------+
|inv_id|acct_id|inv_total|inv_items|acct_age|num_inv_120d|cum_tot_inv_acct|is_late|
+------+-------+---------+---------+--------+------------+----------------+-------+
|     1|      1| 56060.72|       16|    1626|          23|              56|      1|
|     2|      1| 34681.89|        6|    1433|          17|              22|      0|
|     3|      1| 46466.53|       15|    1392|          12|              13|      0|
|     4|      1| 46154.11|       10|    1541|          21|              41|      0|
|     5|      1|120371.88|       21|    1600|          20|              50|      0|
|     6|      1|  2446.93|        8|    1424|          17|              20|      0|
|     7|      1| 55755.93|       16|    1402|          16|              17|      0|
|     8|      1| 34691.34|       15|    1454|          18|              26|      0|
|     9|      1|  4700.64|        5|    1396|          14|              15| 

23/07/30 22:51:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 558991 ms exceeds timeout 120000 ms
23/07/30 22:51:44 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/30 22:51:46 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc

In [ ]:
# STEP 6: SAVE FINAL DATASET TO CSV FILE

invoices_final.write.option("header",True) \
                    .option("delimiter",",") \
                    .csv("data_output", header=True, mode="overwrite")

**----------------------------------------------------------------------------------------------------**
# **DATA MODELS & DESIGN CHOICES**
**----------------------------------------------------------------------------------------------------**

**Data Model:**

- ETL pipeline built using Spark SQL, intermediate data models created to store the results of different stages of data transformation.

- **Intermediate Model:**
    Invoices_total - aggregated summary of invoice items, total quantity of items, total cost price and total retail price for each invoice. The model simplifies the ETL process and provides a cleaner workflow.

    Additional intermediate models may be used in more complex ETL pipelines to enhance performance and enable data validation.


**Design Choices and Scaling Considerations:**

- **Spark SQL:** Using Spark for the ETL pipeline allows us to process large volumes of data efficiently, thanks to its distributed processing capabilities.

- **Scalability:** By partitioning and distributing data across a cluster, Spark can process data in parallel, making it scalable to handle big data workloads.

- **Data Source and Data Lakes:** The pipeline reads data from CSV files as the data source and writes the final features dataset to a CSV file as the data lake. Depending on the production environment, we might use other data sources and lakes like databases and cloud storage.

- **Testing the ETL Job:** To test the ETL job, we can use unit testing frameworks like PySpark's pyspark.sql.test.DataFrameTest, which allows us to compare expected results with the ETL pipeline's output.